In [2]:
from NYTimes import *

In [3]:
import folium, wikipedia

In [4]:
def relatedURLS(article, report=False):
    if 'related_urls' in article:
        #print(article)
        try:
            related = article['related_urls'] 
            facets = {}
            urls = []
            for record in related:
                for item in record:
                    urls.append({item:record[item]})
            for record in article:
                if 'facet' in record:
                    facets[record] = article[record]
        except:
            print('No related URLS, use facets instead')
    return urls, facets
def followURLSMap(layer, report=False):
    report = {}
    x=0
    for i in range(0,len(layer),):
        if 'url' in layer[i].keys():
            try:
                rArticle = specificArticle(layer[i]['url'])
                results = rArticle['results'][0]
                report[x] = results
                x += 1
            except:
                pass
    return report
def layer3Relations(layer1_relations, layer2):
    i=0
    layer1_urls = []
    for x in range(0,len(layer1_relations),):
        if 'url' in layer1_relations[x].keys():
            layer1_urls.append(layer1_relations[x]['url'])
    new_urls = []
    for x in layer2:
        try:
            urls = layer2[x]['related_urls']
            for x in range(0,len(urls)):
                if 'url' in  urls[x].keys():
                    if urls[x]['url'] not in layer1_urls:
                        if urls[x] not in new_urls:
                            new_urls.append(urls[x])
        except:pass
    return new_urls
def makePopUp(results):
    summary = results['abstract']
    article_template = """<b>Title: </b>{}</br>
    <b>Byline:</b> {}</br>
    <b>Abstract</b>: {}</br>
    <b>link:</b> {}""".format(results['title'],
                                   results['byline'], 
                                   results['abstract'], 
                                   results['url'])
    if 'multimedia' in results:
        if results['multimedia'] != '':
            for entry in results['multimedia']:
                if entry['type'] == 'image':
                    media = entry
        article_template+="""</br>
         <img src="{}" alt={}></br>
         <b>Caption: </b>{}</br>
         """.format(media['url'], media['copyright'],media['caption'])
    return article_template
def addCluster(layer, cluster, color):
    for article in layer:
        for place in layer[article]['geo_facet']:
            try:
                coordinate = wikipedia.page(place).coordinates
            except:
                coordinate = (40.45, -73.59)
            sub_article_template = makePopUp(layer[article])
            secondaryPopUp = folium.Html(sub_article_template, script=True)
            articlePopup = folium.Popup(secondaryPopUp, max_width=450,)
            Marker = folium.Marker(location=coordinate,
                                   popup=articlePopup,
                                   icon = folium.Icon(color=color)
                                  ).add_to(cluster)
def dataLayerFromRelated(urls):
    layer_report = {}
    x = 0
    for article in urls:
        try:
            #print(article['url'])
            layer_report[x] = specificArticle(article['url'])['results'][0]
            x+=1
        except:pass
    return layer_report

In [5]:
def mapArticleRelations(article):
    center = (39, 34)
    primaryPost = (40.45, -73.59)
    layer1 = specificArticle(article['url'])
    if layer1['results'][0]['geo_facet'] != 0:
        try:
            primaryPost = wikipedia.page(layer1['results'][0]['geo_facet']).coordinates
        except:
            pass
    newsMap = folium.Map(location=primaryPost,tiles='Stamen Watercolor', zoom_start=2)
    layer2Cluster = folium.MarkerCluster().add_to(newsMap)
    layer3Cluster = folium.MarkerCluster().add_to(newsMap)
    print(layer1['results'][0]['title'])
    print('Primary: Blue')
    print('Layer 2 Relations: Red')
    print('Layer 3 Relations: Green')
    article_template = makePopUp(article)
    primaryPopUp = folium.Html(article_template, script=True)
    popup = folium.Popup(primaryPopUp, max_width=450)
    folium.Marker(primaryPost, popup=popup).add_to(newsMap)
    layer1_relations, layer1_facets = relatedURLS(article)
    layer2 = followURLSMap(layer1_relations, report=False)
    print('layer2  {}'.format(len(layer2)))
    layer3_urls = layer3Relations(layer1_relations, layer2)
    layer3 = dataLayerFromRelated(layer3_urls)
    print('layer3 {}'.format(len(layer3)))
    addCluster(layer2, layer2Cluster, color='red')
    addCluster(layer3, layer3Cluster, color='green')
    layer2Cluster.add_to(newsMap)
    layer3Cluster.add_to(newsMap)
    folium.TileLayer('openstreetmap').add_to(newsMap)
    folium.TileLayer('Stamen Toner').add_to(newsMap)
    folium.TileLayer('mapboxcontrolroom').add_to(newsMap)
    folium.TileLayer('cartodbpositron').add_to(newsMap)
    folium.LayerControl().add_to(newsMap)
    return newsMap

In [12]:
report = newsWire(source='nyt', section='world', limit=10, report=False)

In [13]:
for article in report['results']:
    if article['related_urls'] not in ['', None, False, "None"]:
        mapTarget=article
        break

In [14]:
testMap = mapArticleRelations(article)

As French Election Nears, Le Pen Targets Voters Her Party Once Repelled
Primary: Blue
Layer 2 Relations: Red
Layer 3 Relations: Green
layer2  6
layer3 16


In [15]:
testMap

In [16]:
testMap.save('../Civics/artifacts/newsMap.html')

In [10]:
report = newsWire(source='nyt', section='Health', timePeriod=120, report=False)
for article in report['results']:
    if article['related_urls'] not in ['', None, False, "None"]:
        mapTarget=article
        break
layer1_relations, layer1_facets = relatedURLS(article)
layer2 = followURLSMap(layer1_relations, report=False)
layer3_urls = layer3Relations(layer1_relations, layer2)
layer3 = dataLayerFromRelated(layer3_urls)
testMap = mapArticleRelations(article)

G.O.P.’s Health Care Tightrope Winds Through the Blue-Collar Midwest
Primary: Blue
Layer 2 Relations: Red
Layer 3 Relations: Green
layer2  4
layer3 11


In [11]:
testMap

TODO Troubleshoot errors in other categories